# Introduction to AiiDA

## Table of contents
<ul class="toc-item">
    <li><a href="#PART--1">PART -1</a>
        <ul class="toc-item">
            <li><a href="#AiiDA-data-types">AiiDA data types</a>
                <ul class="toc-item">
                    <li><a href="#Most-common-datatypes">Most common datatypes</a></li>
                    <li><a href="#Base-types">Base types</a>
                        <ul class="toc-item">
                            <li><a href="#Dict">Dict</a></li>
                            <li><a href="#StructureData">StructureData</a></li>
                            <li><a href="#ArrayData">ArrayData</a></li>
                            <li><a href="#FolderData">FolderData</a></li>
                            <li><a href="#SinglefileData">SinglefileData</a></li>
                            <li><a href="#RemoteData">RemoteData</a></li>
                        </ul>
                    </li>
                    <li><a href="#Code">Code</a></li>
                </ul>
            </li>
        </ul>
    </li>
    <li><a href="#PART---2">PART - 2</a>
        <ul class="toc-item">
            <li><a href="#Verdi-Commands">Verdi Commands</a>
                <ul class="toc-item">
                    <li><a href="#Looking-for-calculations">Looking for calculations</a></li>
                    <li><a href="#Looking-at-other-data-structures">Looking at other data structures</a></li>
                    <li><a href="#AiiDA-graph">AiiDA graph</a></li>
                    <li><a href="#Codes-and-computers">Codes and computers</a></li>
                </ul>
            </li>
        </ul>
    </li>
</ul>

# PART -1

## AiiDA data types

There are a number of data types distributed with AiiDA. We summarize here the most common, and some useful features/functionalities to work with them.

### Most common datatypes


Here follows a short summary of common datatypes already provided with AiiDA. This list is not
complete, see also inside the module `aiida.orm.nodes.data` for the list of all available plugins.

### Base types

In the `aiida.orm.nodes.data.base` module there are a number of useful classes
that wrap base python datatypes (like `aiida.orm.nodes.data.int.Int`,
`aiida.orm.nodes.data.float.Float`, `aiida.orm.nodes.data.str.Str`, ...).
These are automatically loaded with the verdi shell, and also directly exposed from `aiida.orm`.
These classes are particularly useful when you need to provide a single parameter to e.g. a
`aiida.engine.processes.functions.workfunction`.

Each of these classes can most often be used transparently (e.g. you can sum two
`aiida.orm.nodes.data.int.Int` objects, etc.). If you need to access the bare
value and not the whole AiiDA class, use the ``.value`` property.

In the same module, there is also a `aiida.orm.nodes.data.list.List` class to
store a list of base data types.

The different datatypes can be accessed through the `aiida.plugins.factories.DataFactory` function
(also exposed from `aiida.plugins`) by passing an entry point to it as an argument. A list of all the data entry points
can be obtain running the command ``verdi plugin list aiida.data``.

#### Dict

* **Class**: `aiida.orm.nodes.data.dict.Dict`
* **String to pass to the** `aiida.plugins.factories.DataFactory`: ``dict``
* **Aim**: store a dictionary of python base types in the database.
  It can store any dictionary where elements can be a base python type (strings, floats,
  integers, booleans, None type, datetime objects) and lists or dictionaries of them, at
  any depth level (e.g. a dictionary where a value is a list of dictionaries of
  strings and floats).
* **What is stored in the database**: all key/values pairs as attributes
* **What is stored in the file repository**: ---
* **Example**:

```python
Dict(dict={'resources'  : {"num_machines": 1},
           'queue_name' : 'blah',
           'max_wallclock_seconds':  60*60,
          }
    )
```

#### StructureData


* **Class**: `aiida.orm.nodes.data.structure.StructureData`
* **String to pass to the** `aiida.plugins.factories.DataFactory`: ``structure``
* **Aim**: store a crystal structure to be used by atomistic codes
* **What is stored in the database**: all atomic positions, species, kinds,
* **What is stored in the file repository**: ---
* **Additional functionality**:
* **Example**:

```python
alat = 4. # angstrom
cell = [[alat, 0., 0.,],
        [0., alat, 0.,],
        [0., 0., alat,],
       ]
s = StructureData(cell=cell)
s.append_atom(position=(0.,0.,0.), symbols='Fe')
s.append_atom(position=(alat/2.,alat/2.,alat/2.), symbols='O')
```

#### ArrayData


* **Class**: `aiida.orm.nodes.data.array.ArrayData`
* **String to pass to the** `~aiida.plugins.factories.DataFactory`: ``array``
* **Aim**: store generic numeric arrays
* **What is stored in the database**: the shape of the arrays and the name of the arrays
* **What is stored in the file repository**: the array data in numpy format

#### FolderData


* **Class**: `aiida.orm.nodes.data.folder.FolderData`
* **String to pass to the** `aiida.plugins.factories.DataFactory`: ``folder``
* **Aim**: store a set of files/folders (with possibly a folder/subfolder structure)
* **What is stored in the database**: ---
* **What is stored in the file repository**: all files and folders

#### SinglefileData

* **Class**: `aiida.orm.nodes.data.singlefile.SinglefileData`
* **String to pass to the** `aiida.plugins.factories.DataFactory`: ``singlefile``
* **Aim**: the same as ``FolderData``, but allows to store only one single file.
* **What is stored in the database**: the filename
* **What is stored in the file repository**: the file

#### RemoteData

* **Class**: `aiida.orm.nodes.data.remote.RemoteData`
* **String to pass to the** `aiida.plugins.factories.DataFactory`: ``remote``
* **Aim**: this basically represents a "symbolic link" to a specific folder on
  a remote computer.
  Its main use is to allow users to persist the provenance when e.g. a calculation
  produces data in a raw/scratch folder, and the whole folder needs to be provided
  to restart/continue.
* **What is stored in the database**: the path of the folder (and the remote computer
  as a `.computer` property, not as an attribute)
* **What is stored in the file repository**: ---

Note: There are many more derived datatypes available in AiiDA, for a fill list see the [AiiDA Documentation](https://aiida-core.readthedocs.io/en/latest/)

### Code

A code represents (in the database) the actual executable used to run the calculation. 
Note that in AiiDA the object `code` in the database is meant to represent a specific executable, i.e. a given compiled version of a code. Every calculation in AiiDA is linked to a code, installed on a specific computer. This means that if you install `fleur` and `inpgen` on two computers A and B, you will need to have two different `codes` in the database (although the source of the code is the same, the binary file is different).

**Example**:

```python
codename = 'fleur@somecomputer'
from aiida.orm import Code
code = Code.get_from_string(codename)
```

<br><br>

# PART - 2

**Note**: All commands starting with a `!` are bash commands. Python notebooks provide this handy way of running shell/bash commands inside a python environment. 

*you can execute the active code cell's by pressing Ctrl+Enter*

*Also read through the comments*

In [ ]:
# example
!ls

## Verdi Commands

In this part of the tutorial you will learn some basics about the AiiDA framework. Get familiar with some useful `verdi` commands.

The command line utility `verdi` is one of the most common ways to interact with AiiDA. Verdi with it's subcommands enables a variety of operations such as inspecting the status of ongoing or terminated calculations, showing the details of calculations, computers, codes, or data structures, access the input and the output of a calculation, etc. Similar to the bash shell, verdi command support Tab completion. Try right now to type verdi in a terminal of the aiida container and tap Tab twice to have a list of subcommands. Whenever you need the explanation of a command type verdi help or add -h flag if you are using any of the verdi subcommands. Finally, fields enclosed in angular brackets, such as `<pk>`, are placeholders to be replaced by the actual value of that field (an integer, a string, etc...)

In [ ]:
# Is the daemon running?
!verdi daemon status

### Looking for calculations

In [ ]:
# List all calculations currently running
!verdi calculation list

In [ ]:
# List all calculations in the Postgres database
!verdi calculation list -a

In [ ]:
# You also say what you would like to see
!verdi calculation list -s FINISHED

In [ ]:
# List all calculations in the database ran during the last 3 days and grep for FAILED calculations 
!verdi calculation list -a -p3 | grep FAILED

In [ ]:
# To look at a calculation in more detail what went wrong, what was parsed in the log, etc,.
!verdi calculation logshow xxx xxx

In [ ]:
# Look at a FINSIHED inpgen calculation in more detail, 
# what nodes went in? what output nodes were produced?
!verdi calculation show xxx

The output depends on the specific `pk` chosen and should inform you about the input nodes (e.g. initial structure, parameters, pseudopotentials, kpoints, etc.), the output nodes (e.g. output structure, output parameters, etc.). For instance, if you choose `pk=xxx`, which identifies a SCF calculation of the Si unit cell obtained using fleur, the output should look like

In [ ]:
# the input file for inpgen for this calculation (with default parameters)
!verdi calculation inputcat xxx

In [ ]:
# the input file for other inpgen calculation (with some parameters) (the one that failed)
!verdi calculation inputcat xxx

In [ ]:
# Now look in more detail at an FLEUR calculation
!verdi calculation show xxx

In [ ]:
# Look at the xml ouput file of the Fleur calculation
!verdi calculation outputcat xxx

In [ ]:
# grep for the distance of that calculation in the out.xml file
!verdi calculation outputcat xxx | grep distance

In [ ]:
# look at the files retrieved from the remote computer/cluster, 
# we store the last charge density (cdn1) too, this way any calculation can be continued from
# from what we have 'locally' in the repository
!verdi calculation outputls xxx

In [ ]:
# Since the out file is also recieved
# You can also grep for the distance of that calculation in the old out file
!verdi calculation outputcat xxx -p out | grep distance

In [ ]:
# look at the output node of that calculation which was saved in the database
# 'output_parameters   376  ParameterData'
!verdi data parameter show xxx

In [ ]:
# We can also get the output node (dictionary) directly with a calculation command
!verdi calculation res xxx

When we start working with AiiDA python framework the same can be achived using the following python code. For example to get the total energy of a calculation:

```python
# If you want to get a result with python code from a caculation you could do so was follows.
calc = load_node(373) # load the calculation node
print('\nTotal energy : {} {}\n'.format(calc.res.energy, calc.res.energy_units))
```

### Looking at other data structures

In [ ]:
# list all cif files in the database 
#(We have downloaded some from the ICSD for you. Then we read them in from a folder)
!verdi data cif list

In [ ]:
# list all crystal structures in the database, find out what pks their nodes are
!verdi data structure list

In [ ]:
!verdi data structure list | grep Si
# better if you look for elements
!verdi data structure list -e Si

In [ ]:
# There are quite some Tungesten compounds
!verdi data structure list -e W

In [ ]:
# AiiDa has also ways to easly display structures (example with xcrysden if installed)
# This does not work here because we do not have x forwarding to the container
!verdi data structure show xxx --format xcrysden

In [ ]:
# but we can export it and you can display it on your laptop if you have xcrysden installed
!verdi data structure export --format xsf xxx > xxx_Si.xsf

### AiiDA graph

AiiDA stores in the database the inputs required by a calculation as well as the its outputs. These objects are connected in a graph that looks like Fig. X for a simple calculation or more like Fig. Y in general.

Inline ascii visualisation of AiiDA graph (nodes in the database and their connection)

**Note**: you can inspect the nodes and test draw them by using the following functions provided by `aiida.orm` and `aiida.utils.ascii_vis`

```python
calc = load_node(xxx)
draw_parents(calc, dist=3) # Is this a head node (no parents)?
draw_children(calc, dist=3) # What are its Children nodes, what was done?
```
or for an inpgen calculation

```python
calc = load_node(xxx)
draw_children(calc, dist=3) # What are its Children nodes, what was done? 
```

Do so also for our Si
```python
draw_children(load_node(xxx), dist=3) # This is not the full graph only to the 3rd connection
```

You can create a similar graph as shown in Fig X for any node by using the utility `verdi graph generate <pk>`. To visualize similar information in graph(ical) form, run (replacing <pk> with xxx):

In [ ]:
#The same can be done more 'visual' for the full graph
#Graph generation (dot file generate)
!verdi graph generate xxx
!verdi graph generate yyy
!verdi graph generate zzz

In [ ]:
# convert the dot file to a pdf and look at it (if you have graphvis installed)
!dot -Tpdf -o xxx.pdf xxx.dot
!dot -Tpdf -o yyy.pdf yyy.dot
!dot -Tpdf -o zzz.pdf zzz.dot

open the pdf files to inspect the provenance graph

### Codes and computers

A code represents (in the database) the actual executable used to run the calculation. Find the `pk` of such a node in the graph and type

In [ ]:
!verdi code show <pk>

The command prints information on the plugin used to interface the code to AiiDA, the remote machine on which the code is executed, the path of its executable, etc. To have a list of all available codes type

In [ ]:
!verdi code list

Among the entries of the output you should also find the code just shown.

Similarly, the list of computers on which AiiDA can submit calculations is accessible by means of the command


In [ ]:
!verdi computer list -a

`-a` shows all computers, also the one imported in your database but that you did not configure, i.e., to which you
don’t have access). Details about each computer can be obtained by the comma

In [ ]:
!verdi computer show <COMPUTERNAME>